# Токенизация текстов в деталях

## TF-IDF

**Term Frequency-Inverse Document Frequency**

Каждый вектор будет характеризовать соответствующий документ на основе встречаемости слов, а каждый элемент вектора — соответствовать какому-то слову из словаря слов всего корпуса. 

Получается матрица размерности `Число документов × Число слов в документах`. Каждому слову присвоен счётчик. Чем ближе число в ячейке к 0, тем менее важно слово в контексте этого документа. `tf-idf = 0` означает, что слово не встречается в тексте вообще.

$$
TFIDF = TF * IDF
$$

$$
TF = \frac{WordCount(w,d)}{Length(d)}
$$

$$
IDF(w,C) = lg\frac{Size(C)}{DocCount(w, C)}
$$

- TF, Term Frequency — частота слова в конкретном документе.

- IDF, Inverse Document Frequency — обратная частота встречаемости слова среди документов.
  
- TFIDF — число, описывающее важность отдельно взятого слова для конкретного документа.

- DocCount — число документов в корпусе С, где встречается слово w;
    
- Size(С) — общее число документов в корпусе C.

Вернёмся к примеру с генетикой. Пусть есть 20 документов про генетику и в каждом из них употребили слово «ген» по 1 разу. При этом слово «мышь» употребили только в одном документе (X) из 1000 слов. Посчитаем для документа X TF_IDF для слов «мышь» и «ген»:
```
TF_ген = 1 / 1000 = 0.001
TF_мышь = 1 / 1000 = 0.001

IDF_ген = log(20 / 20) = 0
IDF_мышь = log(20 / 1) =~ 1.301


TF_IDF_ген = 0.001 * 0 = 0
TF_IDF_мышь = 0.001 * 1.301 = 0.001301
```
Если слово не характерно для корпуса текстов, то, скорее всего, на него стоит обратить больше внимания.

В более общем случае метод TF-IDF работает не только с отдельными словами, но и с n-gram'ами:

***N-gram — последовательность из N-символов или слов, идущих по порядку, получаемых путём токенизации. Словарь в TF-IDF будет состоять из N-gram***

Пример:
```
text = 'мама мыла раму'

# 1-gram, по словам ИЛИ  word-level токенизация
'мама', мыла', 'раму' # единица словаря в этом случае - 1 слово

# 2-gram, по словам
'мама мыла', 'мыла раму' # единица словаря - 2 слова

# 2-gram, посимвольно или  character-level токенизация
'ма', 'ам', 'а ', ' м', 'мы', 'ыл', 'ла', ' р', 'ра', 'му' # обратите внимание на пробелы в составе n-gram 
```

## Предобработка корпуса текстов:

Задача токенизации — разбить текст на отдельные элементы, которые в дальнейшем будут использованы для анализа. Алгоритмами, выполняющими разбиение, то есть токенизаторами, могут быть:

- Регулярные выражения.
   
- Предобученные словари, построенные на большом количестве документов и выучившие наиболее частотное для конкретного языка совместное расположение символов в текстах.

Предобученные токенизаторы включены во многие библиотеки по работе с естественным языком, такие как NLTK (Natural Language ToolKit), spacy и transformers из экосистемы huggingface. 

In [1]:
from collections import Counter
import re

from ml_dl_experiments import settings

with open(settings.SOURCE_PATH + "datasets/tiny_shakespear.txt", "r") as f:
    text = f.read()

text[:60]

'First Citizen:\nBefore we proceed any further, hear me speak.'

In [2]:
# регулярное выражение - оставляет только слова длиной  > 1 символа без пунктуации
pattern = r"(?u)\b\w\w+\b"

re.findall(pattern, text[:60])

['First',
 'Citizen',
 'Before',
 'we',
 'proceed',
 'any',
 'further',
 'hear',
 'me',
 'speak']

Токенизируем этот же фрагмент с помощью разных библиотек и сравним результаты. Используем библиотеки NLTK, transformers, а также библиотеку spacy:

In [10]:
import spacy
from nltk import word_tokenize
from transformers import AutoTokenizer
import nltk

# для некоторых библиотек требуется ручной вызов загрузки
# необходимых словарей, модулей. 
nltk.download('punkt')
nltk.download('punkt_tab')

# для spacy загрузим отдельную модель/словарь для английского языка en_core_web_sm    
nlp = spacy.load(settings.SOURCE_PATH + "ml_dl/models/en_core_web_sm-3.8.0")

# в transformers используем предобученный токенизатор от модели для английского языка
tokenizer = AutoTokenizer.from_pretrained(settings.SOURCE_PATH + "ml_dl/models/distilbert-base-uncased-distilled-squad")

#в nltk указываем язык, с которым работаем явно
tokens_nltk = word_tokenize(text[:60], language='english') 

# для spacy предварительно оборачиваем текст во внутренний формат
# представления документа и используем атрибут .text у токена
doc = nlp(text[:60])
tokens_spacy = [token.text for token in doc]

# токенайзеры transfomers работают с текстами напрямую
tokens_trf = tokenizer.tokenize(text[:60])

print("\nNLTK токены:", tokens_nltk)
print("\ntransformers токены:", tokens_trf )
print("\nspaCy токены:", tokens_spacy) 

[nltk_data] Downloading package punkt to /home/ollldman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ollldman/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



NLTK токены: ['First', 'Citizen', ':', 'Before', 'we', 'proceed', 'any', 'further', ',', 'hear', 'me', 'speak', '.']

transformers токены: ['first', 'citizen', ':', 'before', 'we', 'proceed', 'any', 'further', ',', 'hear', 'me', 'speak', '.']

spaCy токены: ['First', 'Citizen', ':', '\n', 'Before', 'we', 'proceed', 'any', 'further', ',', 'hear', 'me', 'speak', '.']


- nltk (word_tokenize) полностью основан на регулярных выражениях.
  
- Токенизатор из transformers является предобученным и выполняет исключительно разделение на токены по словарю. Он оптимизирован под скорость работы и обработку батчами.
   
- Токенизатор из spacy также предобучался, но содержит намного больше атрибутов для каждого токена (например, можно вызвать token.like_email или like_url). Часть из них вычисляются по заданным правилам (regex), часть предсохранены для каждого токена.

In [14]:
text = "123: hey ds_expert@w.com , check findme.com"

# преобразуйте текст во внутренний формат spacy
doc = nlp(text)
tokens = [token.text for token in doc]

# проверьте, является ли токен числом, имейлом, ссылкой
digits = [token for token in doc if token.is_digit ]
emails = [token for token in doc if token.like_email]
urls = [token for token in doc if token.like_url]

print("токены:", tokens)
print("числа:", digits)
print("имейлы:", emails)
print("ссылки:", urls)

токены: ['123', ':', 'hey', 'ds_expert@w.com', ',', 'check', 'findme.com']
числа: [123]
имейлы: [ds_expert@w.com]
ссылки: [findme.com]


Условно токенизаторы можно разделить по назначению:

- spacy позволяют решать некоторые NLP-задачи даже без применения модели.
  
- transformers рассчитаны на быструю токенизацию с минимальными затратами, принося в жертву набор функций.
  
- nltk — простое разделение на токены регулярками без дополнительных функций.

In [16]:
with open(settings.SOURCE_PATH + "datasets/tiny_shakespear.txt", "r") as f:
    text = f.read()

pattern = r"(?u)\b\w\w+\b"
tokens_regex = re.findall(pattern, text.lower())

cnt = Counter(tokens_regex)
print(cnt.most_common(5))
print(cnt.most_common()[-5:]) 

[('the', 6287), ('and', 5690), ('to', 4934), ('of', 3760), ('you', 3211)]
[('fowling', 1), ('weakly', 1), ('drowsiness', 1), ('possesses', 1), ('eyelids', 1)]


Самые частые токены — это союзы и артикли, которые в общем не несут основной смысловой нагрузки.

Такие слова есть в каждом языке, и в контексте NLP их принято называть «стоп-словами». Как правило, при работе с моделями частотности слов они удаляются, так как не полезны для решения конечной задачи и лишь увеличивают размерность словаря.

Легко удалить такие слова можно с помощью готового набора стоп-слов. Такие наборы существуют в библиотеках spacy и nltk. Давайте на примере библиотеки nltk посмотрим, как выполнить их обработку:

In [17]:
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

tokens_filtered = [t for t in tokens_regex if t not in stop_words] # фильтруем

cnt = Counter(tokens_filtered)
print(cnt.most_common(5))
print(cnt.most_common()[-5:]) 

[('thou', 1421), ('thy', 1059), ('king', 925), ('shall', 849), ('thee', 762)]
[('fowling', 1), ('weakly', 1), ('drowsiness', 1), ('possesses', 1), ('eyelids', 1)]


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ollldman/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


В целом всё получилось, хотя просочились устаревшие формы слов вроде thy и thou. Это говорит о том, что готовые наборы предобработки — хорошо, но всё же стоит валидировать результаты работы глазами, особенно если у вас какая-то нетипичная лексика в тексте.

>***При обучении трансформерных архитектур иногда может потребоваться корректировка исходного текста путём удаления части токенов. Например, обилие гиперссылок в онлайн-чатах может привести к переобучению. При принятии решения об удалении токенов в каждом отдельном случае стоит отталкиваться от целевой задачи и доменной специфики текстов.***

## TF-IDF `scikit-learn`

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

with open(settings.SOURCE_PATH + "datasets/tiny_shakespear.txt", "r") as f:
    text = f.read()

# разобьём текст на части - шаг необязательный, 
# просто искусственно делаем из 1 документа 10 разных)
step = 100000
docs = [text[i:i+step] for i in range(0, len(text), step)]

tf_idf = TfidfVectorizer(
    stop_words=stop_words, # сразу передадим стоп-слова
    max_features=10_000, # размер словаря
    ngram_range=(1,2), # 1-2 словесные n-gramы
    min_df=2, # токен не реже, чем в 2 документах
    max_df=0.95 # не учитываем токен с встречаемостью > 95%
    ) 
# используем комбинацию методов fit и transform
tf_idf_matrix = tf_idf.fit_transform(docs)

print("N_documents x Vocabular_size:", tf_idf_matrix.shape)
print(tf_idf.get_feature_names_out()[:5]) # 5 токенов из словаря
print(tf_idf_matrix[0, :5].todense()) # веса этих 5 токенов

N_documents x Vocabular_size: (12, 10000)
['abate' 'abhor' 'abhorr' 'abhorred' 'abide']
[[0.         0.00361604 0.00259977 0.         0.        ]]


Метод автоматически токенизирует тексты, используя регулярное выражение по умолчанию, однако можно передать свой токенизатор в аргумент `tokenizer`.
`TfidfVectorizer` также содержит несколько интересных параметров:

- `max_features` — позволяет заранее установить размер словаря — отбирает топ самых частотных токенов при превышении max_features. Можно использовать, чтобы повысить производительность: меньше матрица → быстрее считаем.
  
- `max_df, min_df` — числовое ограничение. Могут быть целым числом 2, 3, 4 или долей в диапазоне [0, 1.0]. Помогают учесть частотность встречаемости слов в документах — можно удалять как слишком частые слова в корпусе (специфичные), так и слишком редкие.
    
- `ngram_range` — опция настройки размера n-gram.
    
- `analyzer` — позволяет применять расчёт n_gram посимвольно (char) или по словам (word).

Часто встречается один и тот же вид слова, это указывает на одну из слабостей использования метода tf-idf напрямую — невозможность работы со словоформами 
>***Словоформа — это вариант слова, принимающий различные грамматические формы в зависимости от его использования в предложении, включая изменения по временам, числам, падежам и т. д. Например, для слова «ягнёнок» есть такие словоформы: «ягнёнка», «ягнёнку», «ягнята», «ягнятами» и др.***

TF-IDF для таких задач:

- Классификация текстов — подача в модели: градиентный бустинг, логистическую регрессию и другое.

- Поиск похожих документов — анализ близости векторов.
    
- Выделение ключевых слов — оценка значимости токенов на основе частотности.
   
- Тематическое моделирование — выявление скрытых тем в документах.